# Tutorial [Raytracing em um final de semana](https://raytracing.github.io/books/RayTracingInOneWeekend.html#rays,asimplecamera,andbackground)

Observações: 

- As **etapas 1 e 2** estão implementadas no diretório **Atividade 01**

- A **etapa 3** está implementada em **Atividade 02**

- A **etapa 4.1** está implementada em **Ray.py** no mesmo repositório

## Importações

In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

from Ray import Ray

## 4.2 Sending Rays Into the scene 

In [ ]:
aspect_ratio = 16.0 / 9.0
image_width = 400

image_height = int(image_width / aspect_ratio)
if image_height < 1:
    image_height = 1

viewport_height = 2.0
viewport_width = viewport_height * (image_width / image_height)
# OBS: viewport_width: não está usando a variável aspect_ratio, pois a variável aspect_ratio é a proporção ideal da imagem, porém, arredondamos image_height para inteiro, podendo assim perder um pouco de precisão na proporção da imagem.

# PAREI NA CAMERA...